[View in Colaboratory](https://colab.research.google.com/github/desirajusantosh/DS/blob/master/style_transfer_l13.ipynb)

In [0]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [1]:
import sys
sys.path.append('../')
#!pip install fastai
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install https://github.com/fastai/fastai/archive/master.zip
!pip install opencv-python
!apt update && apt install -y libsm6 libxext6
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision
import torch
from fastai.conv_learner import *
from fastai.dataset import *

from pathlib import Path
import json
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects

    100% |████████████████████████████████| 89.6MB 602kB/s 
    100% |████████████████████████████████| 1.3MB 9.5MB/s 
    100% |████████████████████████████████| 1.5MB 14.1MB/s 
    100% |████████████████████████████████| 112kB 21.0MB/s 
    100% |████████████████████████████████| 174kB 18.2MB/s 
    79% |█████████████████████████▎      | 16.5MB 34.2MB/s eta 0:00:01

    100% |████████████████████████████████| 20.9MB 2.2MB/s 
    100% |████████████████████████████████| 3.4MB 6.1MB/s 
    100% |████████████████████████████████| 112kB 10.1MB/s 
    100% |████████████████████████████████| 51kB 18.3MB/s 
    100% |████████████████████████████████| 51kB 14.9MB/s 
    100% |████████████████████████████████| 2.2MB 12.2MB/s 


    100% |████████████████████████████████| 61kB 19.8MB/s 
    100% |████████████████████████████████| 4.2MB 9.1MB/s 
    100% |████████████████████████████████| 552kB 20.1MB/s 
    100% |████████████████████████████████| 13.7MB 3.9MB/s 
    100% |████████████████████████████████| 92kB 22.3MB/s 
    100% |████████████████████████████████| 5.7MB 8.0MB/s 
    100% |████████████████████████████████| 860kB 22.0MB/s 
    100% |████████████████████████████████| 471kB 21.0MB/s 
    100% |████████████████████████████████| 81kB 28.2MB/s 


    100% |████████████████████████████████| 71kB 18.4MB/s 
  Cache entry deserialization failed, entry ignored
    100% |████████████████████████████████| 102kB 23.0MB/s 
    100% |████████████████████████████████| 61kB 19.9MB/s 
    100% |████████████████████████████████| 81kB 16.3MB/s 
  Running setup.py bdist_wheel for fastai ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-_zb7knhh/wheels/64/3d/9f/d12a217aa2531321c5b9ae96288fcae2687d3b744376e8f94f
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ |

 / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done
  Stored in directory: /content/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
  Running setup.py bdist_wheel for feather-format ... - done
  Stored in directory: /content/.cache/pip/wheels/85/7d/12/2dfa5c0195f921ac935f5e8f27deada74972edc0ae9988a9c1
  Running setup.py bdist_wheel for kaggle-cli ... - done
  Stored in directory: /content/.cache/pip/wheels/d5/bb/10/c1dd1b08c7433c943cb55c46367ae3f891415e8a37300ff8a7
  Running setup.py bdist_wheel for pandas-summary ... - done
  Stored in directory: /content/.cache/pip/wheels/86/a9/6b/36b727a9ec687dac08bd3d0501cddd7b1a223943513eb04a03
  Running setup.py bdist_wheel for torchtext ... - \ done
  Stored in directory: /content/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e56

Get:7 http://security.ubuntu.com/ubuntu artful-security/main amd64 Packages [199 kB]
Get:8 http://archive.ubuntu.com/ubuntu artful-updates/main amd64 Packages [325 kB]
Fetched 980 kB in 2s (462 kB/s)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
All packages are up to date.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6 is already the newest version (2:1.2.2-1).
libxext6 is already the newest version (2:1.3.3-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
kaggle-cli 0.12.13 has requirement lxml<4.1,>=4.0.0, but you'll have lxml 4.2.1 which is incompatible.
plotnine 0.3.0 has requirement scipy>=1.0.0, but you'll have scipy 0.19.1 which is incompatible.
kaggle-cli 0.12.13 has requirement lxml<4.1,>=4.0.0, but you'll have lxml 4.2.1 which is incompatible.


In [0]:
from fastai.conv_learner import *
from pathlib import Path
from scipy import ndimage

In [0]:
torch.backends.cudnn.benchmark=True

In [4]:
ls

datalab/


In [0]:
ls datalab/

In [0]:
mkdir data/imagenet -p

In [7]:
cd data/imagenet/


/content/data/imagenet


In [0]:
!wget -c http://files.fast.ai/data/imagenet-sample-train.tar.gz

--2018-05-01 00:46:40--  http://files.fast.ai/data/imagenet-sample-train.tar.gz
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2232645974 (2.1G) [application/x-tar]
Saving to: ‘imagenet-sample-train.tar.gz’

              image  25%[====>               ] 537.79M  12.9MB/s    eta 2m 24s 

imagenet-sample-tra 100%[===================>]   2.08G  15.1MB/s    in 2m 42s  

2018-05-01 00:49:22 (13.1 MB/s) - ‘imagenet-sample-train.tar.gz’ saved [2232645974/2232645974]

